In [2]:
# ==========================================
# 🧹 LIMPIEZA Y PREPARACIÓN FINAL DEL DATASET DE FARMACIA
# ==========================================

import pandas as pd
import re

# ==============================
# 1️⃣ Cargar dataset unificado
# ==============================
df = pd.read_csv("../output/dataset_farmacia_unificado.csv")
print(f"Dataset original: {df.shape[0]} filas x {df.shape[1]} columnas")

Dataset original: 171867 filas x 17 columnas


In [3]:
# ==============================
# 2️⃣ Conversión robusta de fechas
# ==============================
df['fecha'] = df['fecha'].astype(str).str.strip()
df['fecha'] = df['fecha'].replace(['', 'nan', 'NaT', 'None'], pd.NA)

# Completar fechas incompletas SOLO si son válidas
def completar_fecha(x):
    if pd.isna(x):
        return pd.NA
    if re.match(r"^\d{4}-\d{2}$", x):     # año-mes
        return x + "-01"
    elif re.match(r"^\d{4}$", x):          # solo año
        return x + "-01-01"
    else:
        return x

df['fecha'] = df['fecha'].apply(completar_fecha)

# Convertir usando dayfirst=True (para formato DD/MM/YYYY)
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce', dayfirst=True)

# También convertir fecha_vencimiento si existe
if 'fecha_vencimiento' in df.columns:
    df['fecha_vencimiento'] = pd.to_datetime(df['fecha_vencimiento'], errors='coerce', dayfirst=True)


In [4]:
# ==============================
# 4️⃣ Filtrar valores válidos
# ==============================
if 'precio' in df.columns:
    df = df[df['precio'] > 0]
if 'cantidad' in df.columns:
    df = df[df['cantidad'] >= 0]



In [5]:
# ==============================
# 5️⃣ Crear columna de cantidad total
# ==============================
if 'fraccion' in df.columns:
    df['cantidad_total'] = df['cantidad'] + df['fraccion'].fillna(0)
    df = df.drop(columns=['cantidad', 'fraccion'])
else:
    df['cantidad_total'] = df['cantidad']



In [6]:
# ==============================
# 7️⃣ Filtrar outliers (precio, total, cantidad_total)
# ==============================
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

for col in ['precio', 'total', 'cantidad_total']:
    if col in df.columns:
        df = remove_outliers(df, col)

print(f"Dataset después de limpiar outliers: {df.shape[0]} filas x {df.shape[1]} columnas")


Dataset después de limpiar outliers: 130583 filas x 16 columnas


In [7]:
# ==============================
# 8️⃣ Eliminar columnas ID (no analíticas)
# ==============================
id_cols = [
    'auto_venta', 'auto_detalle', 'venta', 'compra2',
    'auto', 'compra', 'medicamento', 'auto_med'
]
df = df.drop(columns=id_cols, errors='ignore')



In [8]:
# ==============================
# 9️⃣ Verificación final
# ==============================
print("\nTipos de datos finales:")
print(df.dtypes)

print("\nNulos por columna:")
print(df.isnull().sum())

print("\nPrimeras filas:")
print(df.head())

# ==============================
# ✅ Guardar dataset limpio
# ==============================
output_path = "../output/dataset_farmacia_limpio.csv"
df.to_csv(output_path, index=False)

print(f"\nDataset limpio guardado como: {output_path}")
print(f"Forma final: {df.shape[0]} filas x {df.shape[1]} columnas")


Tipos de datos finales:
fecha                datetime64[ns]
monto_recibido              float64
precio                      float64
total                       float64
precio_compra               float64
fecha_vencimiento    datetime64[ns]
nombre                       object
cantidad_total              float64
dtype: object

Nulos por columna:
fecha                78490
monto_recibido           0
precio                   0
total                    0
precio_compra            0
fecha_vencimiento    20075
nombre                   0
cantidad_total           0
dtype: int64

Primeras filas:
       fecha  monto_recibido  precio  total  precio_compra fecha_vencimiento  \
0 2019-07-01           470.0     9.5    9.5           8.13        2020-01-01   
1 2019-07-01           470.0     2.5    5.0           1.50        2020-01-01   
2 2019-07-01           470.0     2.5    5.0           1.75        2019-01-09   
3 2019-07-01           470.0     4.5    4.5           3.70        2020-01-01   
4 2019-